In [1]:
from keras.models import Sequential
from keras.layers import Dense, Dropout

import pandas as pd
import numpy as np
import math

/Users/michaelshen/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")
submit = pd.read_csv('gender_submission.csv')

In [3]:
train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [5]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
PassengerId    891 non-null int64
Survived       891 non-null int64
Pclass         891 non-null int64
Name           891 non-null object
Sex            891 non-null object
Age            714 non-null float64
SibSp          891 non-null int64
Parch          891 non-null int64
Ticket         891 non-null object
Fare           891 non-null float64
Cabin          204 non-null object
Embarked       889 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 83.6+ KB


In [6]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 11 columns):
PassengerId    418 non-null int64
Pclass         418 non-null int64
Name           418 non-null object
Sex            418 non-null object
Age            332 non-null float64
SibSp          418 non-null int64
Parch          418 non-null int64
Ticket         418 non-null object
Fare           417 non-null float64
Cabin          91 non-null object
Embarked       418 non-null object
dtypes: float64(2), int64(4), object(5)
memory usage: 36.0+ KB


In [7]:
train.describe()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,891.000000,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000
mean,446.000000,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208
std,257.353842,0.486592,0.836071,14.526497,1.102743,0.806057,49.693429
min,1.000000,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,223.500000,0.000000,2.000000,20.125000,0.000000,0.000000,7.910400
50%,446.000000,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,668.500000,1.000000,3.000000,38.000000,1.000000,0.000000,31.000000
max,891.000000,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


In [8]:
test.describe()

,PassengerId,Pclass,Age,SibSp,Parch,Fare
count,418.000000,418.000000,332.000000,418.000000,418.000000,417.000000
mean,1100.500000,2.265550,30.272590,0.447368,0.392344,35.627188
std,120.810458,0.841838,14.181209,0.896760,0.981429,55.907576
min,892.000000,1.000000,0.170000,0.000000,0.000000,0.000000
25%,996.250000,1.000000,21.000000,0.000000,0.000000,7.895800
50%,1100.500000,3.000000,27.000000,0.000000,0.000000,14.454200
75%,1204.750000,3.000000,39.000000,1.000000,0.000000,31.500000
max,1309.000000,3.000000,76.000000,8.000000,9.000000,512.329200


In [9]:
selected_features = ['Pclass', 'Age', 'SibSp', 'Parch', 'Fare', 'Sex', 'Embarked']

In [10]:
X_train = train[selected_features]

In [11]:
y_train = train['Survived']

In [12]:
X_test = test[selected_features]

In [13]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 7 columns):
Pclass      891 non-null int64
Age         714 non-null float64
SibSp       891 non-null int64
Parch       891 non-null int64
Fare        891 non-null float64
Sex         891 non-null object
Embarked    889 non-null object
dtypes: float64(2), int64(3), object(2)
memory usage: 48.8+ KB


In [14]:
X_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 7 columns):
Pclass      418 non-null int64
Age         332 non-null float64
SibSp       418 non-null int64
Parch       418 non-null int64
Fare        417 non-null float64
Sex         418 non-null object
Embarked    418 non-null object
dtypes: float64(2), int64(3), object(2)
memory usage: 22.9+ KB


In [15]:
X_train['Age'].fillna(X_train['Age'].mean(), inplace=True)
X_train['Embarked'].fillna('S', inplace=True)
X_test['Age'].fillna(X_test['Age'].mean(), inplace=True)
X_test['Fare'].fillna(X_test['Fare'].mean(), inplace=True)

/Users/michaelshen/anaconda3/lib/python3.6/site-packages/pandas/core/generic.py:3660: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._update_inplace(new_data)


In [16]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 7 columns):
Pclass      891 non-null int64
Age         891 non-null float64
SibSp       891 non-null int64
Parch       891 non-null int64
Fare        891 non-null float64
Sex         891 non-null object
Embarked    891 non-null object
dtypes: float64(2), int64(3), object(2)
memory usage: 48.8+ KB


In [17]:
X_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 7 columns):
Pclass      418 non-null int64
Age         418 non-null float64
SibSp       418 non-null int64
Parch       418 non-null int64
Fare        418 non-null float64
Sex         418 non-null object
Embarked    418 non-null object
dtypes: float64(2), int64(3), object(2)
memory usage: 22.9+ KB


In [18]:
X_train.Embarked = X_train.Embarked.map({'S': 1, 'C': 2, 'Q':3})
X_test.Embarked = X_test.Embarked.map({'S': 1, 'C': 2, 'Q':3})

/Users/michaelshen/anaconda3/lib/python3.6/site-packages/pandas/core/generic.py:3110: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value


In [19]:
X_train.Sex = X_train.Sex.map({'female': 0, 'male': 1})
X_test.Sex = X_test.Sex.map({'female': 0, 'male': 1})

/Users/michaelshen/anaconda3/lib/python3.6/site-packages/pandas/core/generic.py:3110: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value


In [20]:
age = np.zeros(len(X_train))
age[X_train['Age']<20] = 1
age[(X_train['Age']>=20)&(X_train['Age']<65)] = 2
age[(X_train['Age']>=65)] = 3
X_train['Age'] = age

/Users/michaelshen/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


In [22]:
age = np.zeros(len(X_test))
age[X_test['Age']<20] = 1
age[(X_test['Age']>=20)&(X_test['Age']<65)] = 2
age[(X_test['Age']>=65)] = 3
X_test['Age'] = age

/Users/michaelshen/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


In [23]:
fare = np.zeros(len(X_train))
fare[X_train['Fare']<8] = 1
fare[(X_train['Fare']>=8)&(X_train['Fare']<14)] = 2
fare[(X_train['Fare']>=14)&(X_train['Fare']<31)] = 3
fare[(X_train['Fare']>=31)] = 4
X_train['Fare'] = fare

/Users/michaelshen/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [24]:
fare = np.zeros(len(X_test))
fare[X_test['Fare']<8] = 1
fare[(X_test['Fare']>=8)&(X_test['Fare']<14)] = 2
fare[(X_test['Fare']>=14)&(X_test['Fare']<31)] = 3
fare[(X_test['Fare']>=31)] = 4
X_test['Fare'] = fare

/Users/michaelshen/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [25]:
X_train.head()

,Pclass,Age,SibSp,Parch,Fare,Sex,Embarked
0,3,2.0,1,0,1.0,1,1
1,1,2.0,1,0,4.0,0,2
2,3,2.0,0,0,1.0,0,1
3,1,2.0,1,0,4.0,0,1
4,3,2.0,0,0,2.0,1,1


In [26]:
model = Sequential()

In [27]:
model.add(Dense(units = 40, input_dim = 7, 
                kernel_initializer = 'uniform', 
                bias_initializer='zeros', 
                activation = 'relu'))

In [28]:
model.add(Dense(units = 30, 
                kernel_initializer = 'uniform',
                bias_initializer='zeros', 
                activation = 'relu'))

In [29]:
model.add(Dense(units = 1, 
                kernel_initializer = 'uniform',
                activation = 'sigmoid'))

In [30]:
model.compile(loss = 'binary_crossentropy', 
              optimizer = 'adam', 
              metrics=['accuracy'])

In [31]:
model.fit(X_train, y_train, validation_split=0.1, epochs=500, batch_size=30, verbose=2)

Train on 801 samples, validate on 90 samples
Epoch 1/500
 - 1s - loss: 0.6905 - acc: 0.5968 - val_loss: 0.6840 - val_acc: 0.6222
Epoch 2/500
 - 0s - loss: 0.6746 - acc: 0.6155 - val_loss: 0.6548 - val_acc: 0.6222
Epoch 3/500
 - 0s - loss: 0.6422 - acc: 0.6155 - val_loss: 0.6096 - val_acc: 0.6222
Epoch 4/500
 - 0s - loss: 0.6028 - acc: 0.6454 - val_loss: 0.5428 - val_acc: 0.7444
Epoch 5/500
 - 0s - loss: 0.5506 - acc: 0.7303 - val_loss: 0.4812 - val_acc: 0.7889
Epoch 6/500
 - 0s - loss: 0.5100 - acc: 0.7516 - val_loss: 0.4467 - val_acc: 0.8111
Epoch 7/500
 - 0s - loss: 0.4821 - acc: 0.7665 - val_loss: 0.4336 - val_acc: 0.8111
Epoch 8/500
 - 0s - loss: 0.4737 - acc: 0.7840 - val_loss: 0.4263 - val_acc: 0.8222
Epoch 9/500
 - 0s - loss: 0.4654 - acc: 0.7928 - val_loss: 0.4182 - val_acc: 0.8111
Epoch 10/500
 - 0s - loss: 0.4634 - acc: 0.8027 - val_loss: 0.4287 - val_acc: 0.8222
Epoch 11/500
 - 0s - loss: 0.4651 - acc: 0.7903 - val_loss: 0.4318 - val_acc: 0.8111
Epoch 12/500
 - 0s - loss: 0.

Epoch 97/500
 - 0s - loss: 0.4350 - acc: 0.8065 - val_loss: 0.3904 - val_acc: 0.8222
Epoch 98/500
 - 0s - loss: 0.4367 - acc: 0.8002 - val_loss: 0.3949 - val_acc: 0.8333
Epoch 99/500
 - 0s - loss: 0.4309 - acc: 0.8052 - val_loss: 0.3900 - val_acc: 0.8222
Epoch 100/500
 - 0s - loss: 0.4321 - acc: 0.8052 - val_loss: 0.3921 - val_acc: 0.8222
Epoch 101/500
 - 0s - loss: 0.4317 - acc: 0.8127 - val_loss: 0.3918 - val_acc: 0.8333
Epoch 102/500
 - 0s - loss: 0.4299 - acc: 0.8090 - val_loss: 0.3913 - val_acc: 0.8222
Epoch 103/500
 - 0s - loss: 0.4290 - acc: 0.8115 - val_loss: 0.3905 - val_acc: 0.8333
Epoch 104/500
 - 0s - loss: 0.4303 - acc: 0.8090 - val_loss: 0.3879 - val_acc: 0.8222
Epoch 105/500
 - 0s - loss: 0.4293 - acc: 0.8102 - val_loss: 0.3953 - val_acc: 0.8222
Epoch 106/500
 - 0s - loss: 0.4310 - acc: 0.8102 - val_loss: 0.3954 - val_acc: 0.8111
Epoch 107/500
 - 0s - loss: 0.4309 - acc: 0.8040 - val_loss: 0.3921 - val_acc: 0.8333
Epoch 108/500
 - 0s - loss: 0.4319 - acc: 0.8065 - val_lo

Epoch 193/500
 - 0s - loss: 0.4249 - acc: 0.8165 - val_loss: 0.3833 - val_acc: 0.8333
Epoch 194/500
 - 0s - loss: 0.4223 - acc: 0.8102 - val_loss: 0.3830 - val_acc: 0.8333
Epoch 195/500
 - 0s - loss: 0.4229 - acc: 0.8177 - val_loss: 0.3772 - val_acc: 0.8222
Epoch 196/500
 - 0s - loss: 0.4218 - acc: 0.8240 - val_loss: 0.3787 - val_acc: 0.8333
Epoch 197/500
 - 0s - loss: 0.4176 - acc: 0.8265 - val_loss: 0.3807 - val_acc: 0.8333
Epoch 198/500
 - 0s - loss: 0.4204 - acc: 0.8227 - val_loss: 0.3808 - val_acc: 0.8333
Epoch 199/500
 - 0s - loss: 0.4223 - acc: 0.8140 - val_loss: 0.3801 - val_acc: 0.8222
Epoch 200/500
 - 0s - loss: 0.4189 - acc: 0.8227 - val_loss: 0.3836 - val_acc: 0.8333
Epoch 201/500
 - 0s - loss: 0.4186 - acc: 0.8177 - val_loss: 0.3794 - val_acc: 0.8222
Epoch 202/500
 - 0s - loss: 0.4186 - acc: 0.8227 - val_loss: 0.3800 - val_acc: 0.8444
Epoch 203/500
 - 0s - loss: 0.4197 - acc: 0.8190 - val_loss: 0.3889 - val_acc: 0.8333
Epoch 204/500
 - 0s - loss: 0.4218 - acc: 0.8152 - val

Epoch 289/500
 - 0s - loss: 0.4096 - acc: 0.8290 - val_loss: 0.3825 - val_acc: 0.8333
Epoch 290/500
 - 0s - loss: 0.4086 - acc: 0.8327 - val_loss: 0.3739 - val_acc: 0.8333
Epoch 291/500
 - 0s - loss: 0.4091 - acc: 0.8290 - val_loss: 0.3761 - val_acc: 0.8333
Epoch 292/500
 - 0s - loss: 0.4063 - acc: 0.8340 - val_loss: 0.3737 - val_acc: 0.8333
Epoch 293/500
 - 0s - loss: 0.4058 - acc: 0.8277 - val_loss: 0.3767 - val_acc: 0.8444
Epoch 294/500
 - 0s - loss: 0.4119 - acc: 0.8265 - val_loss: 0.3744 - val_acc: 0.8333
Epoch 295/500
 - 0s - loss: 0.4068 - acc: 0.8352 - val_loss: 0.3746 - val_acc: 0.8333
Epoch 296/500
 - 0s - loss: 0.4071 - acc: 0.8327 - val_loss: 0.3750 - val_acc: 0.8444
Epoch 297/500
 - 0s - loss: 0.4053 - acc: 0.8327 - val_loss: 0.3770 - val_acc: 0.8444
Epoch 298/500
 - 0s - loss: 0.4041 - acc: 0.8365 - val_loss: 0.3724 - val_acc: 0.8444
Epoch 299/500
 - 0s - loss: 0.4104 - acc: 0.8302 - val_loss: 0.3700 - val_acc: 0.8444
Epoch 300/500
 - 0s - loss: 0.4056 - acc: 0.8327 - val

Epoch 385/500
 - 0s - loss: 0.3898 - acc: 0.8377 - val_loss: 0.3671 - val_acc: 0.8444
Epoch 386/500
 - 0s - loss: 0.3946 - acc: 0.8340 - val_loss: 0.3670 - val_acc: 0.8444
Epoch 387/500
 - 0s - loss: 0.3900 - acc: 0.8365 - val_loss: 0.3671 - val_acc: 0.8444
Epoch 388/500
 - 0s - loss: 0.3927 - acc: 0.8352 - val_loss: 0.3692 - val_acc: 0.8444
Epoch 389/500
 - 0s - loss: 0.3937 - acc: 0.8315 - val_loss: 0.3752 - val_acc: 0.8222
Epoch 390/500
 - 0s - loss: 0.3935 - acc: 0.8352 - val_loss: 0.3682 - val_acc: 0.8556
Epoch 391/500
 - 0s - loss: 0.3905 - acc: 0.8302 - val_loss: 0.3700 - val_acc: 0.8444
Epoch 392/500
 - 0s - loss: 0.3892 - acc: 0.8352 - val_loss: 0.3709 - val_acc: 0.8222
Epoch 393/500
 - 0s - loss: 0.3930 - acc: 0.8340 - val_loss: 0.3685 - val_acc: 0.8444
Epoch 394/500
 - 0s - loss: 0.3939 - acc: 0.8315 - val_loss: 0.3694 - val_acc: 0.8444
Epoch 395/500
 - 0s - loss: 0.3945 - acc: 0.8315 - val_loss: 0.3704 - val_acc: 0.8444
Epoch 396/500
 - 0s - loss: 0.3913 - acc: 0.8315 - val

Epoch 481/500
 - 0s - loss: 0.3777 - acc: 0.8340 - val_loss: 0.3761 - val_acc: 0.8333
Epoch 482/500
 - 0s - loss: 0.3791 - acc: 0.8365 - val_loss: 0.3727 - val_acc: 0.8222
Epoch 483/500
 - 0s - loss: 0.3820 - acc: 0.8365 - val_loss: 0.3753 - val_acc: 0.8222
Epoch 484/500
 - 0s - loss: 0.3802 - acc: 0.8377 - val_loss: 0.3731 - val_acc: 0.8222
Epoch 485/500
 - 0s - loss: 0.3804 - acc: 0.8340 - val_loss: 0.3748 - val_acc: 0.8333
Epoch 486/500
 - 0s - loss: 0.3787 - acc: 0.8390 - val_loss: 0.3759 - val_acc: 0.8111
Epoch 487/500
 - 0s - loss: 0.3814 - acc: 0.8377 - val_loss: 0.3825 - val_acc: 0.8222
Epoch 488/500
 - 0s - loss: 0.3811 - acc: 0.8377 - val_loss: 0.3774 - val_acc: 0.8333
Epoch 489/500
 - 0s - loss: 0.3765 - acc: 0.8427 - val_loss: 0.3822 - val_acc: 0.8222
Epoch 490/500
 - 0s - loss: 0.3893 - acc: 0.8302 - val_loss: 0.3802 - val_acc: 0.8222
Epoch 491/500
 - 0s - loss: 0.3878 - acc: 0.8340 - val_loss: 0.3757 - val_acc: 0.8333
Epoch 492/500
 - 0s - loss: 0.3808 - acc: 0.8427 - val

In [33]:
scores = model.evaluate(X_train, y_train)
scores[1]

891/891 [==============================] - 0s 22us/step


0.8417508419515308

In [34]:
survived_predict = model.predict(X_test)
survived_predict = np.array(survived_predict)
survived_predict = np.round(survived_predict)
survived_predict

array([[0.],
       [0.],
       [0.],
       [0.],
       [1.],
       [0.],
       [1.],
       [0.],
       [1.],
       [0.],
       [0.],
       [0.],
       [1.],
       [0.],
       [1.],
       [1.],
       [0.],
       [0.],
       [0.],
       [1.],
       [1.],
       [1.],
       [1.],
       [0.],
       [1.],
       [0.],
       [1.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [1.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [1.],
       [1.],
       [0.],
       [0.],
       [0.],
       [1.],
       [1.],
       [0.],
       [0.],
       [1.],
       [1.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [1.],
       [0.],
       [0.],
       [0.],
       [1.],
       [1.],
       [1.],
       [1.],
       [0.],
       [0.],
       [1.],
       [1.],
       [0.],
       [1.],
       [0.],
       [1.],
       [0.],
       [0.],

In [35]:
submission=pd.DataFrame({'PassengerId':test['PassengerId'], 'Survived':survived_predict[0:418,0]})

In [36]:
submission.to_csv('submission.csv', index=False)